In [ ]:
pip install catboost

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
from google.colab import drive

# Google Drive'ı bağlayın
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Covid Data.csv" , sep=",", encoding='utf-8')

In [ ]:
df.shape

(1048575, 21)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   USMER                 1048575 non-null  int64 
 1   MEDICAL_UNIT          1048575 non-null  int64 
 2   SEX                   1048575 non-null  int64 
 3   PATIENT_TYPE          1048575 non-null  int64 
 4   DATE_DIED             1048575 non-null  object
 5   INTUBED               1048575 non-null  int64 
 6   PNEUMONIA             1048575 non-null  int64 
 7   AGE                   1048575 non-null  int64 
 8   PREGNANT              1048575 non-null  int64 
 9   DIABETES              1048575 non-null  int64 
 10  COPD                  1048575 non-null  int64 
 11  ASTHMA                1048575 non-null  int64 
 12  INMSUPR               1048575 non-null  int64 
 13  HIPERTENSION          1048575 non-null  int64 
 14  OTHER_DISEASE         1048575 non-null  int64 
 15

In [ ]:
df.isnull().sum()

USMER                   0
MEDICAL_UNIT            0
SEX                     0
PATIENT_TYPE            0
DATE_DIED               0
INTUBED                 0
PNEUMONIA               0
AGE                     0
PREGNANT                0
DIABETES                0
COPD                    0
ASTHMA                  0
INMSUPR                 0
HIPERTENSION            0
OTHER_DISEASE           0
CARDIOVASCULAR          0
OBESITY                 0
RENAL_CHRONIC           0
TOBACCO                 0
CLASIFFICATION_FINAL    0
ICU                     0
dtype: int64

In [ ]:
df.PREGNANT = df.PREGNANT.replace(97, 2) #Replace 97 with not pregnant.
df.PREGNANT = df.PREGNANT.replace(98, 2)

In [ ]:
f = df[(df.PNEUMONIA == 1) | (df.PNEUMONIA == 2)]
df = df[(df.DIABETES == 1) | (df.DIABETES == 2)]
df = df[(df.COPD == 1) | (df.COPD == 2)]
df = df[(df.ASTHMA == 1) | (df.ASTHMA == 2)]
df = df[(df.INMSUPR == 1) | (df.INMSUPR == 2)]
df = df[(df.HIPERTENSION == 1) | (df.HIPERTENSION == 2)]
df = df[(df.OTHER_DISEASE == 1) | (df.OTHER_DISEASE == 2)]
df = df[(df.CARDIOVASCULAR == 1) | (df.CARDIOVASCULAR == 2)]
df = df[(df.OBESITY == 1) | (df.OBESITY == 2)]
df = df[(df.RENAL_CHRONIC == 1) | (df.RENAL_CHRONIC == 2)]
df = df[(df.TOBACCO == 1) | (df.TOBACCO == 2)]

In [ ]:
print(df['ICU'].value_counts())
print(df['INTUBED'].value_counts())

ICU
97    844063
2     173154
1      16434
99      7362
Name: count, dtype: int64
INTUBED
97    844063
2     156712
1      33034
99      7204
Name: count, dtype: int64


In [ ]:
df.drop(['ICU', 'INTUBED'], axis=1, inplace=True)

In [ ]:
df['DEATH'] = [2 if row=='9999-99-99' else 1 for row in df['DATE_DIED']]

In [ ]:
df.drop('DATE_DIED', axis=1, inplace=True)

In [ ]:

X = df.drop('DEATH', axis=1)
y = df['DEATH']

In [ ]:
sampler = RandomUnderSampler(random_state=42)
X_sampled, y_sampled = sampler.fit_resample(X, y)
print(y_sampled.value_counts())

DEATH
1    75420
2    75420
Name: count, dtype: int64


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1041013 entries, 0 to 1048574
Data columns (total 19 columns):
 #   Column                Non-Null Count    Dtype
---  ------                --------------    -----
 0   USMER                 1041013 non-null  int64
 1   MEDICAL_UNIT          1041013 non-null  int64
 2   SEX                   1041013 non-null  int64
 3   PATIENT_TYPE          1041013 non-null  int64
 4   PNEUMONIA             1041013 non-null  int64
 5   AGE                   1041013 non-null  int64
 6   PREGNANT              1041013 non-null  int64
 7   DIABETES              1041013 non-null  int64
 8   COPD                  1041013 non-null  int64
 9   ASTHMA                1041013 non-null  int64
 10  INMSUPR               1041013 non-null  int64
 11  HIPERTENSION          1041013 non-null  int64
 12  OTHER_DISEASE         1041013 non-null  int64
 13  CARDIOVASCULAR        1041013 non-null  int64
 14  OBESITY               1041013 non-null  int64
 15  RENAL_CHRONIC       

In [ ]:
scaler = MinMaxScaler()

# Veri setinizdeki sayısal sütunları seçin ve ölçeklendirin
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_sampled, y_sampled, test_size=0.2, random_state=42)
print('X train :', X_train.shape)
print('X test :', X_test.shape)
print('y train :', y_train.shape)
print('y test :', y_test.shape)

X train : (120672, 18)
X test : (30168, 18)
y train : (120672,)
y test : (30168,)


In [ ]:
def scaling(data):
    scaler = StandardScaler()
    data['AGE'] = scaler.fit_transform(data.loc[:,['AGE']])
    return data

X_train_scaled = scaling(X_train)
X_test_scaled = scaling(X_test)


In [ ]:
def metric(algorithm_name, y_true, preds):


    print(f'Model Ismı {algorithm_name}')
    print('='*50)
    cm = confusion_matrix(y_true, preds)
    F1 = f1_score(y_true, preds)
    total = sum(sum(cm))
    accuracy = (cm[0,0] + cm[1,1])/total
    sensitivity = cm[0,0]/(cm[0,0] + cm[0,1])
    specificity = cm[1,1]/(cm[1,0]+cm[1,1])
    print(f'\nThe confusion Matrix  :\n{cm}')
    print(f'\nThe F1 Score          : {F1}')
    print(f'\nThe accuracy          : {accuracy}')
    print(f'\nThe sensitivity       : {sensitivity}')
    print(f'\nThe specificity       : {specificity}')
    print('='*50)


In [ ]:
logistic = LogisticRegression(max_iter=1000)
logistic.fit(X_train_scaled, y_train)
preds_lr = logistic.predict(X_test_scaled)
metric('Logistic Regression', y_test, preds_lr)

Model Ismı Logistic Regression

The confusion Matrix  :
[[13683  1365]
 [ 1752 13368]]

The F1 Score          : 0.8977462848144867

The accuracy          : 0.896678599840891

The sensitivity       : 0.9092902711323764

The specificity       : 0.8841269841269841


In [ ]:
Naive_Bayes = GaussianNB()
Naive_Bayes.fit(X_train_scaled,y_train)
preds_Bayes = Naive_Bayes.predict(X_test_scaled)
metric('GaussianNB', y_test, preds_Bayes)

Model Ismı GaussianNB

The confusion Matrix  :
[[13042  2006]
 [ 2107 13013]]

The F1 Score          : 0.8637944166639071

The accuracy          : 0.8636634844868735

The sensitivity       : 0.8666932482721956

The specificity       : 0.8606481481481482


In [ ]:
KNN = KNeighborsClassifier(n_neighbors=13,metric="minkowski")
KNN.fit(X_train_scaled, y_train)
preds_KNN = KNN.predict(X_test_scaled)
metric('KNN', y_test, preds_KNN)

Model Ismı KNN

The confusion Matrix  :
[[14254   794]
 [ 1878 13242]]

The F1 Score          : 0.9143040410519564

The accuracy          : 0.911429329090427

The sensitivity       : 0.9472355130249868

The specificity       : 0.8757936507936508


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Modellerin tanımlanması
models = {
    "KNN": KNeighborsClassifier(n_neighbors=13, metric="minkowski"),
    "DecisionTree": DecisionTreeClassifier(max_depth=5, max_features=4),
    "RandomForest": RandomForestClassifier(),
    "GradientBoosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "LogisticRegression": LogisticRegression(max_iter=10),
    "SGDClassifier": SGDClassifier(),
    "SVM": SVC(),
    "LinearSVC": LinearSVC(),
    "GaussianNB": GaussianNB()
}

# Model listesinin oluşturulması
model_list = []

# Modellerin eğitilmesi ve çıktılarının alınması
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    model_list.append({
        "Name": name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    })

# Model sonuçlarının gösterilmesi
for model_result in model_list:
    print(model_result)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


{'Name': 'KNN', 'Accuracy': 0.911429329090427, 'Precision': 0.883585420282668, 'Recall': 0.9472355130249868, 'F1 Score': 0.9143040410519564}
{'Name': 'DecisionTree', 'Accuracy': 0.8698289578361177, 'Precision': 0.8602526724975704, 'Recall': 0.8823763955342903, 'F1 Score': 0.8711740970376931}
{'Name': 'RandomForest', 'Accuracy': 0.9013192787059136, 'Precision': 0.8806205461310462, 'Recall': 0.9279638490164805, 'F1 Score': 0.9036725448956479}
{'Name': 'GradientBoosting', 'Accuracy': 0.9138491116414744, 'Precision': 0.8860633877070024, 'Recall': 0.9493620414673046, 'F1 Score': 0.9166212184402169}
{'Name': 'AdaBoost', 'Accuracy': 0.9015513126491647, 'Precision': 0.895896158384686, 'Recall': 0.9081605528973951, 'F1 Score': 0.9019866675466967}
{'Name': 'LogisticRegression', 'Accuracy': 0.8170578095995757, 'Precision': 0.7986585595185859, 'Recall': 0.8466905901116427, 'F1 Score': 0.8219734847262993}
{'Name': 'SGDClassifier', 'Accuracy': 0.8850106072659772, 'Precision': 0.8647388647388647, 'Re

In [ ]:

results_df = pd.DataFrame(model_list)

# DataFrame'in model ismine göre sıralanması
results_df = results_df.set_index("Name")

# Sonuçların gösterilmesi

styled_df = results_df.style.background_gradient(cmap='coolwarm')
styled_df

,Accuracy,Precision,Recall,F1 Score
Name,,,,
KNN,0.911429,0.883585,0.947236,0.914304
DecisionTree,0.869829,0.860253,0.882376,0.871174
RandomForest,0.901319,0.880621,0.927964,0.903673
GradientBoosting,0.913849,0.886063,0.949362,0.916621
AdaBoost,0.901551,0.895896,0.908161,0.901987
LogisticRegression,0.817058,0.798659,0.846691,0.821973
SGDClassifier,0.885011,0.864739,0.912148,0.887811
SVM,0.890977,0.872710,0.914872,0.893294
LinearSVC,0.891740,0.877193,0.910420,0.893498


CrossValid

In [ ]:
from sklearn.model_selection import cross_validate

CrossValidResult = []

# Her model için çapraz doğrulama sonuçlarını hesaplayalım
for model in models.values():
    cv_results = cross_validate(model, X, y, cv=5,
                                scoring={'precision': 'precision',
                                         'recall': 'recall',
                                         'f1': 'f1',
                                         'accuracy': 'accuracy',
                                         'roc_auc': 'roc_auc',
                                         'average_precision': 'average_precision',
                                         'neg_brier_score': 'neg_brier_score'})

    # Çapraz doğrulama sonuçlarını bir sözlük olarak saklayalım
    model_results = {
        'Model': type(model).__name__,
        'Precision': cv_results['test_precision'].mean(),
        'Recall': cv_results['test_recall'].mean(),
        'F1': cv_results['test_f1'].mean(),
        'Accuracy': cv_results['test_accuracy'].mean(),
        'ROC AUC': cv_results['test_roc_auc'].mean(),
        'Average Precision': cv_results['test_average_precision'].mean(),
        'Neg Brier Score': cv_results['test_neg_brier_score'].mean()
    }

    CrossValidResult.append(model_results)

# Sonuçları DataFrame'e çevirerek yazdıralım
CrossValidResult_df = pd.DataFrame(CrossValidResult)

# Sonuçların gösterilmesi ve arka plan renklendirmesi
styled_cv_df = CrossValidResult_df.style.background_gradient(cmap='coolwarm')
styled_cv_df